In [ ]:
from pathlib import Path
import pandas as pd
import re

def clean_column_name(name: str) -> str:
    name = name.strip()
    if name.lower().startswith("x") and name[1:2].isdigit():
        name = name[1:]  # Strip leading 'X'
    return re.sub(r"[^0-9a-zA-Z]+", "_", name.strip().lower())

def clean_all_numeric_columns(df):
    # Loop through all columns
    for col in df.columns:
        # Try to convert column to numeric after removing commas
        # But only if the column is of type object (strings), since numeric columns won't need this
        if df[col].dtype == 'object':
            # Remove commas
            no_commas = df[col].str.replace(",", "", regex=False)
            # Try converting to numeric (coerce errors to NaN)
            converted = pd.to_numeric(no_commas, errors='coerce')

            # If conversion produced some valid numeric data (not all NaNs), replace original
            if converted.notna().sum() > 0:
                df[col] = converted
    return df

def preprocess_season_data(season_df):
    season_df = season_df.drop(
        columns=["Average.Time.of.Possession.per.Game", "Time.of.Possession"], errors="ignore"
    )
    season_df = clean_all_numeric_columns(season_df)
    if "Win-Loss" in season_df.columns:
        season_df["Win-Loss"] = season_df["Win-Loss"].astype(str).str.strip()
        split_wins_losses = season_df["Win-Loss"].str.split("-", expand=True)
        season_df["Win"] = pd.to_numeric(split_wins_losses[0], errors="coerce").fillna(0).astype(int)
        season_df["Loss"] = pd.to_numeric(split_wins_losses[1], errors="coerce").fillna(0).astype(int)
        season_df = season_df.drop(columns=["Win-Loss"])
    season_df.columns = [clean_column_name(col) for col in season_df.columns]
    season_df.columns = [col.lower() for col in season_df.columns]
    return season_df

def preprocess_game_data(game_df, name_mapping):
    game_df = game_df[game_df["season_type"].str.lower() != "postseason"]
    # Replace team names
    game_df["home_team"] = game_df["home_team"].replace(name_mapping)
    game_df["away_team"] = game_df["away_team"].replace(name_mapping)
    valid_teams = set(name_mapping.values())
    game_df = game_df[
        (game_df["home_team"].isin(valid_teams)) &
        (game_df["away_team"].isin(valid_teams))
    ]
    game_df.columns = [clean_column_name(col) for col in game_df.columns]
    game_df.columns = [col.lower() for col in game_df.columns]
    return game_df

def merge_season_with_games(season_df, game_df):
    # Prepare home and away season data with prefixes
    team_col = season_df[['team']]
    home_season_data = pd.concat([team_col, season_df.drop(columns=['team']).add_prefix('home_')], axis=1)
    away_season_data = pd.concat([team_col, season_df.drop(columns=['team']).add_prefix('away_')], axis=1)

    merged = game_df.merge(
        home_season_data,
        how="left",
        left_on="home_team",
        right_on="team"
    ).drop(columns=["team"])

    merged = merged.merge(
        away_season_data,
        how="left",
        left_on="away_team",
        right_on="team"
    ).drop(columns=["team"])

    # Fill missing season stats with 0 and add missing indicators
    home_cols = [col for col in merged.columns if col.startswith('home_')]
    away_cols = [col for col in merged.columns if col.startswith('away_')]

    merged['home_stats_missing'] = merged[home_cols].isnull().any(axis=1).astype(int)
    merged['away_stats_missing'] = merged[away_cols].isnull().any(axis=1).astype(int)

    merged[home_cols] = merged[home_cols].fillna(0)
    merged[away_cols] = merged[away_cols].fillna(0)

    merged["winner"] = (merged["home_points"] > merged["away_points"]).astype(int)


    return merged

# --- Your existing team name mapping here ---
name_mapping = {
    "Air Force (Mountain West)": "Air Force",
    "Akron (MAC)": "Akron",
    "Alabama (SEC)": "Alabama",
    "App State (Sun Belt)": "Appalachian State",
    "Appalachian St. (Sun Belt)": "Appalachian State",
    "Arizona (Pac-12)": "Arizona",
    "Arizona St. (Pac-12)": "Arizona State",
    "Arkansas (SEC)": "Arkansas",
    "Arkansas St. (Sun Belt)": "Arkansas State",
    "Army West Point (FBS Independent)": "Army",
    "Auburn (SEC)": "Auburn",
    "Ball St. (MAC)": "Ball State",
    "Baylor (Big 12)": "Baylor",
    "Boise St. (Mountain West)": "Boise State",
    "Boston College (ACC)": "Boston College",
    "Bowling Green (MAC)": "Bowling Green",
    "Buffalo (MAC)": "Buffalo",
    "BYU (FBS Independent)": "BYU",
    "BYU (Big 12)": "BYU",
    "California (Pac-12)": "California",
    "Central Mich. (MAC)": "Central Michigan",
    "Charlotte (C-USA)": "Charlotte",
    "Charlotte (AAC)": "Charlotte",
    "Cincinnati (AAC)": "Cincinnati",
    "Cincinnati (Big 12)": "Cincinnati",
    "Clemson (ACC)": "Clemson",
    "Coastal Carolina (Sun Belt)": "Coastal Carolina",
    "Colorado (Pac-12)": "Colorado",
    "Colorado St. (Mountain West)": "Colorado State",
    "Duke (ACC)": "Duke",
    "East Carolina (AAC)": "East Carolina",
    "Eastern Mich. (MAC)": "Eastern Michigan",
    "FIU (C-USA)": "FIU",
    "FIU (CUSA)": "FIU",
    "Fla. Atlantic (C-USA)": "Florida Atlantic",
    "Fla. Atlantic (AAC)": "Florida Atlantic",
    "Florida (SEC)": "Florida",
    "Florida St. (ACC)": "Florida State",
    "Fresno St. (Mountain West)": "Fresno State",
    "Ga. Southern (Sun Belt)": "Georgia Southern",
    "Georgia (SEC)": "Georgia",
    "Georgia St. (Sun Belt)": "Georgia State",
    "Georgia Tech (ACC)": "Georgia Tech",
    "Houston (AAC)": "Houston",
    "Houston (Big 12)": "Houston",
    "Illinois (Big Ten)": "Illinois",
    "Indiana (Big Ten)": "Indiana",
    "Iowa (Big Ten)": "Iowa",
    "Iowa St. (Big 12)": "Iowa State",
    "Kansas (Big 12)": "Kansas",
    "Kansas St. (Big 12)": "Kansas State",
    "Kent St. (MAC)": "Kent State",
    "Kentucky (SEC)": "Kentucky",
    "Liberty (FBS Independent)": "Liberty",
    "Liberty (CUSA)": "Liberty",
    "Louisiana (Sun Belt)": "Louisiana",
    "Louisiana Tech (C-USA)": "Louisiana Tech",
    "Louisiana Tech (CUSA)": "Louisiana Tech",
    "Louisville (ACC)": "Louisville",
    "LSU (SEC)": "LSU",
    "Marshall (Sun Belt)": "Marshall",
    "Marshall (C-USA)": "Marshall",
    "Maryland (Big Ten)": "Maryland",
    "Massachusetts (FBS Independent)": "Massachusetts",
    "Memphis (AAC)": "Memphis",
    "Miami (FL) (ACC)": "Miami",
    "Miami (OH) (MAC)": "Miami (OH)",
    "Michigan (Big Ten)": "Michigan",
    "Michigan St. (Big Ten)": "Michigan State",
    "Middle Tenn. (C-USA)": "Middle Tennessee",
    "Middle Tenn. (CUSA)": "Middle Tennessee",
    "Minnesota (Big Ten)": "Minnesota",
    "Mississippi St. (SEC)": "Mississippi State",
    "Missouri (SEC)": "Missouri",
    "Navy (AAC)": "Navy",
    "NC State (ACC)": "NC State",
    "Nebraska (Big Ten)": "Nebraska",
    "Nevada (Mountain West)": "Nevada",
    "New Mexico (Mountain West)": "New Mexico",
    "North Carolina (ACC)": "North Carolina",
    "North Texas (C-USA)": "North Texas",
    "North Texas (AAC)": "North Texas",
    "Northwestern (Big Ten)": "Northwestern",
    "Notre Dame (ACC)": "Notre Dame",
    "Notre Dame (FBS Independent)": "Notre Dame",
    "Ohio (MAC)": "Ohio",
    "Ohio St. (Big Ten)": "Ohio State",
    "Oklahoma (Big 12)": "Oklahoma",
    "Oklahoma St. (Big 12)": "Oklahoma State",
    "Ole Miss (SEC)": "Ole Miss",
    "Oregon (Pac-12)": "Oregon",
    "Oregon St. (Pac-12)": "Oregon State",
    "Penn St. (Big Ten)": "Penn State",
    "Pittsburgh (ACC)": "Pittsburgh",
    "Purdue (Big Ten)": "Purdue",
    "Rice (C-USA)": "Rice",
    "Rice (AAC)": "Rice",
    "Rutgers (Big Ten)": "Rutgers",
    "San Diego St. (Mountain West)": "San Diego State",
    "San Jose St. (Mountain West)": "San Jose State",
    "SMU (AAC)": "SMU",
    "South Alabama (Sun Belt)": "South Alabama",
    "South Carolina (SEC)": "South Carolina",
    "South Fla. (AAC)": "South Florida",
    "Southern California (Pac-12)": "Southern California",
    "Southern Miss. (C-USA)": "Southern Mississippi",
    "Southern Miss. (Sun Belt)": "Southern Mississippi",
    "Stanford (Pac-12)": "Stanford",
    "Syracuse (ACC)": "Syracuse",
    "TCU (Big 12)": "TCU",
    "Temple (AAC)": "Temple",
    "Tennessee (SEC)": "Tennessee",
    "Texas (Big 12)": "Texas",
    "Texas A&M (SEC)": "Texas A&M",
    "Texas St. (Sun Belt)": "Texas State",
    "Texas Tech (Big 12)": "Texas Tech",
    "Toledo (MAC)": "Toledo",
    "Troy (Sun Belt)": "Troy",
    "Tulane (AAC)": "Tulane",
    "Tulsa (AAC)": "Tulsa",
    "UAB (C-USA)": "UAB",
    "UAB (AAC)": "UAB",
    "UCF (AAC)": "UCF",
    "UCF (Big 12)": "UCF",
    "UCLA (Pac-12)": "UCLA",
    "UNLV (Mountain West)": "UNLV",
    "Utah (Pac-12)": "Utah",
    "Utah St. (Mountain West)": "Utah State",
    "UTEP (C-USA)": "UTEP",
    "UTEP (CUSA)": "UTEP",
    "UTSA (C-USA)": "UTSA",
    "UTSA (AAC)": "UTSA",
    "Vanderbilt (SEC)": "Vanderbilt",
    "Virginia (ACC)": "Virginia",
    "Virginia Tech (ACC)": "Virginia Tech",
    "Wake Forest (ACC)": "Wake Forest",
    "Washington (Pac-12)": "Washington",
    "Washington St. (Pac-12)": "Washington State",
    "West Virginia (Big 12)": "West Virginia",
    "Western Ky. (C-USA)": "Western Kentucky",
    "Western Ky. (CUSA)": "Western Kentucky",
    "Western Mich. (MAC)": "Western Michigan",
    "Wisconsin (Big Ten)": "Wisconsin",
    "Wyoming (Mountain West)": "Wyoming"
}

# Example: list your seasons and game files in matching pairs,
# where each season file matches with the following year games file
season_files = ["cfb19.csv","cfb20.csv","cfb21.csv"]  # add as many as you have
game_files = ["games2020.csv","games2021.csv","games2022.csv"]  # next year games

all_merged = []  # to accumulate dataframes for each season+games pair

for season_file, game_file in zip(season_files, game_files):
    print(f"Processing {season_file} + {game_file}")

    season_data = pd.read_csv(season_file)
    season_data = preprocess_season_data(season_data)
    season_data["team"] = season_data["team"].replace(name_mapping)

    game_data = pd.read_csv(game_file)
    game_data = preprocess_game_data(game_data, name_mapping)

    merged_df = merge_season_with_games(season_data, game_data)

    all_merged.append(merged_df)

# Concatenate all merged dataframes into one large dataframe
final_merged = pd.concat(all_merged, ignore_index=True)

home = final_merged[[
    "home_team", "season", "week", "home_points", "away_points", "id", "start_date"
]].copy()
home["team"] = home["home_team"]
home["points_scored"] = home["home_points"]
home["points_allowed"] = home["away_points"]
home["home_away"] = "home"

away = final_merged[[
    "away_team", "season", "week", "home_points", "away_points", "id", "start_date"
]].copy()
away["team"] = away["away_team"]
away["points_scored"] = away["away_points"]
away["points_allowed"] = away["home_points"]
away["home_away"] = "away"

long_df = pd.concat([home, away], ignore_index=True)

# Sort and calculate rolling stats
long_df = long_df.sort_values(by=["team", "season", "start_date"])

long_df["rolling_avg_points_scored"] = (
    long_df.groupby(["team", "season"])["points_scored"]
    .shift(1)
    .rolling(window=3, min_periods=2)
    .mean()
)

long_df["rolling_avg_points_allowed"] = (
    long_df.groupby(["team", "season"])["points_allowed"]
    .shift(1)
    .rolling(window=3, min_periods=2)
    .mean()
)

long_df["win"] = (long_df["points_scored"] > long_df["points_allowed"]).astype(int)
long_df["rolling_win_pct"] = (
    long_df.groupby(["team", "season"])["win"]
    .shift(1)
    .rolling(window=3, min_periods=2)
    .mean()
)

# Separate home and away rolling stats
home_rolling = long_df[long_df["home_away"] == "home"][[
    "id", "rolling_avg_points_scored", "rolling_avg_points_allowed", "rolling_win_pct"
]].rename(columns={
    "rolling_avg_points_scored": "home_rolling_avg_points_scored",
    "rolling_avg_points_allowed": "home_rolling_avg_points_allowed",
    "rolling_win_pct": "home_rolling_win_pct"
})

away_rolling = long_df[long_df["home_away"] == "away"][[
    "id", "rolling_avg_points_scored", "rolling_avg_points_allowed", "rolling_win_pct"
]].rename(columns={
    "rolling_avg_points_scored": "away_rolling_avg_points_scored",
    "rolling_avg_points_allowed": "away_rolling_avg_points_allowed",
    "rolling_win_pct": "away_rolling_win_pct"
})

# Merge rolling stats back into final_merged on game id
final_merged = final_merged.merge(home_rolling, on="id", how="left")
final_merged = final_merged.merge(away_rolling, on="id", how="left")

cols_to_drop = [
    "home_unnamed_0",
    "home_na",
    "home_time_of_possession",
    "home_average_time_of_possession_per_game",
    "away_unnamed_0",
    "away_na",
    "away_time_of_possession",
    "away_average_time_of_possession_per_game",
    "home_opp_field_goals_made",
    "home_feild_goals",
    "away_opp_field_goals_made",
    "away_field_goals",
    "home_opp_feild_goals_made",
    "home_feild_goals",
    "home_field_goals",
    "away__40",
    "home__40",
    "away_opp_feild_goals_made",
    "away_feild_goals",
]

final_merged = final_merged.drop(columns=[col for col in cols_to_drop if col in final_merged.columns])
print("Number of columns in final_merged:", final_merged.shape[1])
# Save to CSV
final_merged.to_csv("check_the_columns.csv", index=False)
print("Done! Saved to final_merged_all_seasons.csv")

Processing cfb23.csv + games2024.csv


<ipython-input-5-a7936451ac42>:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  season_df["Win"] = pd.to_numeric(split_wins_losses[0], errors="coerce").fillna(0).astype(int)
<ipython-input-5-a7936451ac42>:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  season_df["Loss"] = pd.to_numeric(split_wins_losses[1], errors="coerce").fillna(0).astype(int)
<ipython-input-5-a7936451ac42>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


Number of columns in final_merged: 318
Done! Saved to final_merged_all_seasons.csv
